In [59]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

import pickle
import pandas as pd
import numpy as np
import json
import random

In [53]:
with open(f'katana-assistant-model.pkl', 'rb') as f:
    model = pickle.load(f)
data = pickle.load( open( "katana-assistant-data.pkl", "rb" ) )
words = data['words']
classes = data['classes']

In [54]:
with open('Data1') as json_data:
    data = json.load(json_data)

In [81]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [85]:
p = bow("Load bood pessure for patient", words)
print (p)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]


In [86]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
   
    results = model.predict([input_data])[0]
    result_index= np.argmax(results)
    tag= classes[result_index]
    for tg in data['intents']:
        if tg['tag']==tag:
            responses= tg['responses']
    
        
        
    # filter out predictions below a threshold, and provide intent index
    #results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    #results.sort(key=lambda x: x[1], reverse=True)
    #return_list = []
#     for r in results:
#         return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    print(tag)
    print(random.choice(responses))
    #return return_list

In [91]:
classify_local('how r u')

greeting
Good to see you again


In [88]:
classes

['adverse_drug',
 'blood_pressure',
 'blood_pressure_search',
 'goodbye',
 'greeting',
 'hospital_search',
 'options',
 'pharmacy_search',
 'thanks']

In [90]:
from flask import Flask, Blueprint,render_template

hello = Blueprint('hello', __name__)

@hello.route('/')
def home():    
    return render_template("index.html")
@hello.route('/get', methods=['POST'])
def get_classify():
    ERROR_THRESHOLD = 0.25
    
    sentence=request.form['user_input']
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    # filter out predictions below a threshold
    result_index= np.argmax(results)
    tag= classes[result_index]
    for tg in data['intents']:
        if tg['tag']==tag:
            responses= tg['responses']
    response=random.choice(responses)
    print(response)
    return render_template('index.html',user_input=sentence,bot_response=response)
    #return response




# running REST interface, port=5000 for direct test, port=5001 for deployment from PM2
if __name__ == "__main__":
    app = Flask(__name__)
    app.register_blueprint(hello, url_prefix='/')

    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Dec/2019 15:39:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2019 15:39:10] "POST /get HTTP/1.1" 200 -


Bye! Come back again soon.


127.0.0.1 - - [02/Dec/2019 15:39:26] "POST /get HTTP/1.1" 200 -


Hi there, how can I help?


In [15]:
# from flask import Flask, Blueprint,render_template

# hello = Blueprint('hello', __name__)


# @hello.route('/')
# def home():    
#     return render_template("home.html")
# @hello.route('/get')
# def get_hello_world():
#     return 'Hello, World!'

# if __name__ == '__main__':
#     app = Flask(__name__)
#     app.register_blueprint(hello, url_prefix='/')

#     app.run()